In [2]:
from pdf_gene import MoonCalc ,generate_pdf
from datetime import datetime
import requests
from tqdm import tqdm


In [3]:
def final_df(date,month, year, path="D:/code/Data_2035", dst='D:/Output'):
    
    date_obj = datetime.strptime(date, "%d-%m-%Y")    
    # Convert to YYYY-MM-DD format
    converted_date = date_obj.strftime("%Y-%m-%d")
    Moon = MoonCalc(path,converted_date,month,year +" AH",dst)
    return Moon.calculate()
final_df = final_df("30-03-2025","SHAWWAL","1446")

In [4]:
df = final_df
# Split 'STATION(Sunset)' into 'Station' and 'SunsetTime'
df[['Station', 'SunsetTime']] = df['STATION(Sunset)'].str.extract(r'^(.*?)\s*\((.*?)\)$')

# Optional: drop the original column if no longer needed
df.drop(columns=['STATION(Sunset)'], inplace=True)

# Reorder columns if desired
df = df[['Station', 'SunsetTime', 'LAG TIME(Minutes)', 'MOON ALTITUDE(Degrees)',
         'SUN_AZIMUTH(Degrees)', 'DAZ(Degrees)', 'ELONGATION(Degrees)', 
         'ILLUMINATION(%)', 'CRITERION']]



In [ ]:
row = df
row.columns = [
    'Station', 'SunsetTime', 'LagTime', 'MoonAltitude', 'SunAzimuth',
    'DAZ', 'Elongation', 'Illumination', 'Criterion'
]



# ✅ Function to format the prompt for each row
def make_prompt(row):
    return (
        f"On {row.name}, at station {row['Station']}, "
        f"with a lag time of {row['LagTime']} minutes, "
        f"moon altitude {row['MoonAltitude']}°, "
        f"sun azimuth {row['SunAzimuth']}°, "
        f"DAZ {row['DAZ']}°, "
        f"elongation {row['Elongation']}°, "
        f"illumination {row['Illumination']}%, "
        f"and visibility criterion '{row['Criterion']}', "
        f"explain whether the moon is likely to be visible and why."
    )


# ✅ Function to query LLaMA 3.2 via Ollama
def query_ollama(prompt, model="llama3.2"):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": model, "prompt": prompt, "stream": False}
    )
    return response.json()["response"].strip()

tqdm.pandas()  # This enables tqdm integration with pandas

df['llama_response'] = df.progress_apply(lambda row: query_ollama(make_prompt(row)), axis=1)
print(df[['Station', 'llama_response']].head())



  0%|                                                                                           | 0/13 [00:00<?, ?it/s]

In [ ]:
# Sample DataFrame
# df = pd.DataFrame({
#     'name': ['Alice', 'Bob'],
#     'product': ['laptop', 'smartphone'],
#     'issue': ['screen not working', 'battery draining fast']
# })

df = final_df
# Generate a response per row
def generate_response(row):
    prompt = (
        f"Customer {row['name']} reported an issue with their {row['product']}: "
        f"'{row['issue']}'. Write a helpful and professional support message."
    )
    response = requests.post(
    'http://localhost:11434/api/generate',
    json={
        "model": "llama3.2",
        "prompt": prompt,
        "stream": False
    }
    )
    return response.json()

# Apply model to DataFrame
df['llama_response'] = df.apply(generate_response, axis=1)

# Show results
print(df[['name', 'llama_response']])

In [ ]:
import pandas as pd


dfd = df

# Unpack the dictionary column
df_expanded = pd.json_normalize(dfd["llama_response"])

# Concatenate the unpacked data with the original DataFrame
df_new = pd.concat([dfd, df_expanded], axis=1).drop('llama_response', axis=1)

# Display the new DataFrame
print(df_new)

In [ ]:
df_new["response"]

In [ ]:
import pandas as pd
from llama_cpp import Llama

# Load the model (use the exact filename and path)
llm = Llama(model_path="llama-3.2-1B.Q4_K_M.gguf", n_ctx=2048)

# Sample DataFrame
df = pd.DataFrame({
    'name': ['Alice', 'Bob'],
    'product': ['laptop', 'smartphone'],
    'issue': ['screen not working', 'battery draining fast']
})

# Generate a response per row
def generate_response(row):
    prompt = (
        f"Customer {row['name']} reported an issue with their {row['product']}: "
        f"'{row['issue']}'. Write a helpful and professional support message."
    )
    output = llm(prompt, max_tokens=200)
    return output["choices"][0]["text"].strip()

# Apply model to DataFrame
df['llama_response'] = df.apply(generate_response, axis=1)

# Show results
print(df[['name', 'llama_response']])


In [ ]:
generate_pdf("30-03-2025", "SHAWWAL","1446")